In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#mperformance etrics
command_validity_rate: float = 0.0 # Valid command rate
hallucination_rate: float = 0.0 # Invalid command rate
safety_trigger_rate: float # Collision prevention rate 
task_completion_rate: float

